In [1]:
#Assginment 1: Fast Trajectory Replanning
#CS 440: Intro to Artificial Intelligence
#Due date: 26th Feburary 2018
#authors: Ayush Joshi, Nikhil Mathur, and Dan Snyder
import heapq
import pygame
import random
import os
import math
import time
from random import *
from random import randint
from random import randrange
from ast import literal_eval as make_tuple
from IPython.core.debugger import set_trace

In [2]:
#Convention for the Environment
# Initialize Grid
# 0 is blocked and 1 is unblocked for track_maze
# 0 is unvisited and 1 is visited for track_maze
# gn = get neighbour
# nn = new neighbour
# p = parent

In [3]:
# Initialize Variables

blockwidth = 8  # Drawing dimensions of block
GridCols = 101   # No of columns
GridRows = 101   # No of rows

maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
track_maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
first_parent_x = 0
first_parent_y = 0
last_parent_x = 0
last_parent_y = 0
cursor_x = 0
cursor_y = 0
i = 0

#main tree
tree = {}

In [4]:
#get the string for the cell location
def get_String(x,y):
    return str(x)+","+str(y)

#get the coordinates for the cell.
def get_coord(cell):
    li = cell.split(",")
    return int(li[0]),int(li[1])

#check if the cell is visited or not.
def is_visited(x,y):
    if x is None and y is None:
        return False
    x= int(x)
    y=int(y)
    if track_maze[x][y] == 1:
        return True
    return False

def is_first(x,y):
    if x == first_parent_x and y == first_parent_y:
        return True
    else:
        return False

# get the key for the value from the tree
def get_key(value):
    key_value_dict = {}
    for i in tree.items():
        a = i[0]
        b = i[1]
        if a and b:
            for item in b:
                key_value_dict[str(item)] = str(a)
    return key_value_dict[value]

In [5]:
def valid_coordinates(x,y):
    if x is None and y is None:
        return False
    if 0<= x <GridCols and 0<= y <GridRows:
        return True
    return False
    
#get the neighbours for a cell.
def get_neighbour(x,y,point):
    x= int(x)
    y= int(y)
    point = int(point)
    if point == 0 and valid_coordinates(x-1,y):
        return x-1,y
    if point == 1 and valid_coordinates(x+1,y):
        return x+1,y
    if point == 2 and valid_coordinates(x,y-1):
        return x,y-1
    if point == 3 and valid_coordinates(x,y+1):
        return x,y+1
    return None,None

#check if the cell has neighbours that are not visited.
def validate_neighbour(x,y):
    x=int(x)
    y=int(y)
    tf_value = False    
        
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            tf_value = True
    except:
        pass  
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            tf_value = True
    except:
        pass
        
    return tf_value

#get the neighbour for a cell
def get_validate_neighbour(x,y):
    x = int(x)
    y = int(y)
    
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            return x-1, y
    except:
        pass
    
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            return x+1, y
    except:
        pass
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            return x, y-1
    except:
        pass
    
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            return x, y+1
    except:
        pass
    
    return None, None


In [6]:
#build_tree returns last child which was visited
def build_tree(p_x,p_y):
    nn_x = None
    nn_y = None
    last_p_x = p_x
    last_p_y = p_y

    while validate_neighbour(p_x,p_y) is True:
        ra = randint(0,3)
        nn_x,nn_y = get_neighbour(p_x,p_y,ra)
        if nn_x is None or nn_y is None or is_visited(nn_x,nn_y):
            while validate_neighbour(p_x,p_y):
                ran = randint(0,3)
                nn_x,nn_y = get_neighbour(p_x,p_y,ran)
                if valid_coordinates(nn_x,nn_y):
                    if is_visited(nn_x,nn_y) == False:
                        break

        if valid_coordinates(nn_x,nn_y) and is_visited(nn_x,nn_y) == False:
            tree[get_String(p_x,p_y)] = [get_String(nn_x,nn_y)] 
            track_maze[int(nn_x)][int(nn_y)] = 1
            last_p_x = nn_x
            last_p_y = nn_y
            rand_block_unblock = randint(0,100)
            #if rand_block_unblock < 30:
                #track_maze[nn_x][nn_y] = 1 # to be changed to 0, using to track
            #else:
                #track_maze[nn_y][nn_y] = 1
            p_x,p_y = nn_x,nn_y
    return last_p_x, last_p_y


In [7]:
#back_track returns new child for a parent with unvisited cell
def back_track(last_x,last_y):
    child_x = last_x
    child_y = last_y
    while is_first(child_x,child_y) == False:
        parent = get_key(get_String(child_x,child_y))
        parent_x,parent_y = get_coord(parent)
        if validate_neighbour(parent_x,parent_y):
            nnn_x,nnn_y = get_validate_neighbour(parent_x,parent_y)
            if nnn_x is not None and nnn_y is not None and is_visited(nnn_x,nnn_y) == False:
                try:
                    newlist = []
                    for item in tree[get_String(parent_x,parent_y)]:
                        newlist.append(item)    
                    newlist.append(get_String(int(nnn_x),int(nnn_y)))
                    #update the list
                    tree[get_String(parent_x,parent_y)] = newlist
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                except:
                    tree[get_String(parent_x,parent_y)] = [get_String(int(nnn_x),int(nnn_y))]
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                return get_String(int(nnn_x),int(nnn_y))
        child_x,child_y = parent_x,parent_y
    return None

In [8]:
# Draw Screen
def drawScreen(GridSurface):
    GameScreen.blit(GridSurface,(0,0))
    pygame.display.update()

In [9]:
# Draw Grid
def drawGrid(myGridSurface):
    myGridSurface.fill((255,255,255))
    for x in range(len(track_maze)):
        for y in range(len(track_maze[x])):
            if track_maze[x][y] == 0: 
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
            elif track_maze[x][y] == 1: 
                pygame.draw.rect(myGridSurface, (255,255,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
    myGridSurface = myGridSurface.convert()
    return myGridSurface

In [10]:
# Make the grid
#areacoordinates = makeGrid()

In [11]:
## For visual
pygame.init()
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())
myfont = pygame.font.SysFont("monospace", 15)
start_x,start_y,goal_x,goal_y = 0,0,0,0
#generateStartFinish()

In [12]:
# Make Random Grid Visual
def makeGrid_visual():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    print("first parent",first_parent_x,first_parent_y)
    print("first child", gn_x,gn_y)
    track_maze[int(p_x)][int(p_y)] = 1
    track_maze[int(gn_x)][int(gn_y)] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    while True:
        #add space for text later
        GameScreen = pygame.display.set_mode((GridCols*blockwidth+20,GridRows*blockwidth+20))
        GridSurface = pygame.Surface(GameScreen.get_size())
        GridSurface = drawGrid(GridSurface)
        
        final_path = []
        closed_list = []
        cell_costs = []
        priority_list = []
        heuristic_list = []
        path_cost = 0
        elapsed_time = 0
        drawmode = 0
        nodes_expanded = 0
        #print(GridSurface)
        drawScreen(GridSurface)
        #time.sleep(2)
        #pygame.quit()
        #i = i +1
        #print("Tree",tree)
        #print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        last_parent_x = new_x
        last_parent_y = new_y
        print("back to main",new_x,new_y)
        if new_x == first_parent_x and new_y == first_parent_y:
            return None
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                return None
    return None


In [13]:
# Make Random Grid Visual
def makeGrid(visual):
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    track_maze[p_x][p_y] = 1
    track_maze[gn_x][gn_y] = 1
    
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    while True:
        if visual:
            GameScreen = pygame.display.set_mode((GridCols*blockwidth+20,GridRows*blockwidth+20))
            GridSurface = pygame.Surface(GameScreen.get_size())
            GridSurface = drawGrid(GridSurface)
            drawScreen(GridSurface)
        
        new_x,new_y = build_tree(p_x,p_y)
        print("build returns",new_x,new_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        
        last_parent_x = new_x
        last_parent_y = new_y
        
        if new_x == first_parent_x and new_y == first_parent_y:
            break
        
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                break
    GameScreen = pygame.display.set_mode((GridCols*blockwidth+20,GridRows*blockwidth+20))
    GridSurface = pygame.Surface(GameScreen.get_size())
    GridSurface = drawGrid(GridSurface)
    drawScreen(GridSurface)
    return None


In [14]:
makeGrid(True)
running = True
while(running):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                pygame.quit()
            
   

first
build returns 36 82
Back track found 37,84
build returns 48 82
Back track found 50,84
build returns 67 67
Back track found 67,69
build returns 51 66
Back track found 50,65
build returns 50 64
Back track found 51,62
build returns 51 61
Back track found 49,63
build returns 46 62
Back track found 44,62
build returns 49 62
Back track found 49,58
build returns 40 57
Back track found 38,57
build returns 42 60
Back track found 42,63
build returns 44 63
Back track found 43,64
build returns 35 54
Back track found 36,52
build returns 31 50
Back track found 33,49
build returns 24 49
Back track found 25,48
build returns 26 47
Back track found 28,47
build returns 30 48
Back track found 22,48
build returns 21 48
Back track found 22,50
build returns 22 50
Back track found 24,53
build returns 35 65
Back track found 36,64
build returns 37 62
Back track found 35,62
build returns 31 61
Back track found 32,60
build returns 29 63
Back track found 28,58
build returns 25 53
Back track found 26,54
build

build returns 87 45
Back track found 83,46
build returns 83 46
Back track found 84,48
build returns 76 54
Back track found 76,58
build returns 77 58
Back track found 76,60
build returns 75 60
Back track found 78,55
build returns 80 56
Back track found 77,53
build returns 75 50
Back track found 82,51
build returns 83 52
Back track found 85,52
build returns 85 49
Back track found 86,51
build returns 86 51
Back track found 84,40
build returns 84 39
Back track found 84,36
build returns 82 40
Back track found 82,38
build returns 82 37
Back track found 79,40
build returns 69 44
Back track found 70,43
build returns 70 43
Back track found 70,45
build returns 71 46
Back track found 73,47
build returns 75 48
Back track found 77,47
build returns 77 45
Back track found 76,46
build returns 76 46
Back track found 78,46
build returns 78 47
Back track found 79,48
build returns 79 49
Back track found 72,48
build returns 68 46
Back track found 67,45
build returns 67 38
Back track found 69,39
build retur

build returns 61 47
Back track found 60,45
build returns 60 45
Back track found 63,48
build returns 65 46
Back track found 66,50
build returns 67 51
Back track found 68,49
build returns 70 51
Back track found 72,50
build returns 72 51
Back track found 71,53
build returns 64 54
Back track found 66,53
build returns 67 53
Back track found 67,55
build returns 64 58
Back track found 69,59
build returns 72 55
Back track found 74,56
build returns 74 56
Back track found 73,57
build returns 73 58
Back track found 71,57
build returns 70 58
Back track found 71,61
build returns 69 60
Back track found 70,53
build returns 70 53
Back track found 74,53
build returns 74 53
Back track found 63,56
build returns 63 56
Back track found 60,55
build returns 60 52
Back track found 58,51
build returns 57 52
Back track found 57,54
build returns 58 54
Back track found 55,52
build returns 54 51
Back track found 57,47
build returns 56 48
Back track found 55,49
build returns 55 49
Back track found 53,43
build retur

build returns 72 71
Back track found 72,69
build returns 72 67
Back track found 73,66
build returns 74 65
Back track found 69,66
build returns 70 68
Back track found 69,65
build returns 69 64
Back track found 67,63
build returns 66 63
Back track found 73,72
build returns 73 72
Back track found 74,68
build returns 74 68
Back track found 78,69
build returns 78 69
Back track found 63,60
build returns 62 62
Back track found 56,60
build returns 53 62
Back track found 55,59
build returns 56 59
Back track found 47,48
build returns 45 49
Back track found 43,50
build returns 43 50
Back track found 42,48
build returns 43 48
Back track found 44,47
build returns 45 47
Back track found 45,45
build returns 44 45
Back track found 43,44
build returns 43 42
Back track found 46,45
build returns 46 45
Back track found 48,46
build returns 49 46
Back track found 40,47
build returns 38 47
Back track found 40,48
build returns 40 48
Back track found 38,49
build returns 37 50
Back track found 35,50
build retur

build returns 18 59
Back track found 13,58
build returns 13 58
Back track found 15,56
build returns 14 56
Back track found 9,60
build returns 8 59
Back track found 7,58
build returns 5 58
Back track found 6,57
build returns 6 57
Back track found 7,61
build returns 5 69
Back track found 5,71
build returns 4 71
Back track found 6,71
build returns 5 72
Back track found 7,73
build returns 20 65
Back track found 19,64
build returns 20 63
Back track found 22,64
build returns 23 64
Back track found 22,68
build returns 19 67
Back track found 20,69
build returns 34 74
Back track found 31,73
build returns 31 70
Back track found 29,70
build returns 28 68
Back track found 26,68
build returns 23 70
Back track found 24,69
build returns 27 70
Back track found 30,75
build returns 25 71
Back track found 24,72
build returns 23 77
Back track found 21,77
build returns 19 70
Back track found 18,73
build returns 8 73
Back track found 10,73
build returns 13 76
Back track found 12,75
build returns 12 74
Back 

Back track found 58,82
build returns 57 80
Back track found 61,80
build returns 60 78
Back track found 64,78
build returns 65 78
Back track found 63,77
build returns 63 77
Back track found 60,71
build returns 60 71
Back track found 58,72
build returns 57 74
Back track found 56,73
build returns 56 72
Back track found 56,78
build returns 56 76
Back track found 57,84
build returns 57 88
Back track found 59,90
build returns 60 90
Back track found 57,96
build returns 57 97
Back track found 56,96
build returns 56 96
Back track found 55,94
build returns 55 93
Back track found 50,78
build returns 50 79
Back track found 60,67
build returns 58 66
Back track found 62,66
build returns 65 68
Back track found 63,68
build returns 62 67
Back track found 61,69
build returns 61 69
Back track found 65,65
build returns 65 65
Back track found 52,92
build returns 51 93
Back track found 46,81
build returns 46 81
Back track found 43,84
build returns 44 87
Back track found 40,84
build returns 39 84
Back track 

In [ ]:
# Main loop
running = True
pygame.init()
myfont = pygame.font.SysFont("monospace", 15)
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())

time.sleep(2)
GridSurface = drawGrid(GridSurface)
start_x,start_y,goal_x,goal_y = generateStartFinish()
final_path = []
closed_list = []
cell_costs = []
priority_list = []
heuristic_list = []
path_cost = 0
elapsed_time = 0
drawmode = 0
nodes_expanded = 0

MySearch = AStarSearch() # Initialize Object

while(running):
	# Get Input
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		elif event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				running = False
			elif event.key == pygame.K_g:
				track_maze = [['1' for y in range(GridRows)] for x in range(GridCols)]
				areacoordinates = makeGrid()
				GridSurface = drawGrid(GridSurface)
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new map")
			elif event.key == pygame.K_e:
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new start and finish points")
			elif event.key == pygame.K_s:
				# Save map: get filename
				filename = raw_input("Save map to: ")
				with open(os.path.join("./gen",filename),"w") as mapfile:
					mapfile.write(str((start_x,start_y)))		# Write start
					mapfile.write("\n")
					mapfile.write(str((goal_x,goal_y)))			# Write goal
					mapfile.write("\n")
					
					for area in areacoordinates:				# Write hard to traverse area centers
						mapfile.write(str((area[0],area[1])))		
						mapfile.write("\n")
					
					for y in range(len(track_maze[x])):				# Write each cell
						for x in range(len(track_maze)):					
							mapfile.write(str(track_maze[x][y]))		
						mapfile.write("\n")
					
					mapfile.close()
				print("Map saved!")
			elif event.key == pygame.K_l:
				# Load map: get filename
				filename = raw_input("Load map from: ")
				with open(os.path.join("./gen",filename),"r") as mapfile: #changed to allow using /maps folder
					new_start = make_tuple(mapfile.readline())
					start_x = new_start[0]
					start_y = new_start[1]
					new_goal = make_tuple(mapfile.readline())
					goal_x = new_goal[0]
					goal_y = new_goal[1]
					
					areacoordinates = []
					
					for i in range(8):
						new_area = make_tuple(mapfile.readline())
						areacoordinates.append((new_area[0],new_area[1]))
					
					for y in range(len(track_maze[x])):				# Read each cell
						for x in range(len(track_maze)):					
							track_maze[x][y] = mapfile.read(1)
						mapfile.read(1)
					
					mapfile.close()
				final_path = []
				closed_list = []
				cell_costs = []
				priority_list = []
				heuristic_list = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				GridSurface = drawGrid(GridSurface)
				print("Map loaded!")
			elif event.key == pygame.K_UP:
				if cursor_y-1 >= 0:
					cursor_y -= 1
			elif event.key == pygame.K_LEFT:
				if cursor_x-1 >= 0:
					cursor_x -= 1
			elif event.key == pygame.K_RIGHT:
				if cursor_x+1 < GridCols:
					cursor_x += 1
			elif event.key == pygame.K_DOWN:
				if cursor_y+1 < GridRows:
					cursor_y += 1
			elif event.key == pygame.K_v:
				# draw closed list
				if drawmode == 0:
					drawmode = 1
				else:
					drawmode = 0
			elif event.key == pygame.K_a:		# -------- A* Search --------
				choice = -1
				while int(choice) < 1 or int(choice) > 6:
					choice = raw_input ("Enter (1) for Manhattan distance, (2) for Euclidean distance, (3) for Octile distance, (4) for Chebyshev distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				MySearch = AStarSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,choice)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_u:		# -------- Uniform Cost Search --------
				MySearch = UniformCostSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,1)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_w:		# -------- Weighted A* Search --------
				choice = -1 #heuristic choice
				weight = 0 #weight of heuristic

				while (int(choice) < 1 or int(choice) > 6):
					choice = raw_input("Enter (1) for Manhattan distance, (2) for Euclidean Distance, (3) for Octile Distance, (4) for Chebyshev Distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				while float(weight) < 1:
					weight = raw_input("Enter the selected weight for Weighted A* - must be >= 1: ")
				
				MySearch = WeightedAStarSearch(weight)
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y, choice)
				elapsed_time = time.time() - start_time
				
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_q:		# -------- Sequential A* Search --------
				MySearch = AStarSearch()

				
		#print(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
		drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)

pygame.quit()